In [1]:
from carsus import init_db
session = init_db("sqlite://")
session.commit()

Initializing the database
Ingesting basic atomic data


In [2]:
from carsus.io.nist import NISTWeightsCompIngester
ingester = NISTWeightsCompIngester()
ingester.download()
ingester.ingest(session)

Ingesting atomic weights


In [3]:
from carsus.io.output import AtomsDataset, Dataset

# Create a dataset and load with the atoms data from the database. 

atoms = AtomsDataset()
atoms.load_sql(session)
print atoms.data.head(5)


              symbol       name  group  period
atomic_number                                 
1                  H   Hydrogen      1       1
2                 He     Helium     18       1
3                 Li    Lithium      1       2
4                 Be  Beryllium      2       2
5                  B      Boron     13       2


In [4]:
# Include atomic weights into the dataset

atoms2 = AtomsDataset()
atoms2.load_sql(session, load_atomic_weights=True)
print atoms2.data.head(5)

              symbol       name  group  period  atomic_weight_value  \
atomic_number                                                         
1                  H   Hydrogen      1       1             1.007975   
2                 He     Helium     18       1             4.002602   
3                 Li    Lithium      1       2             6.967500   
4                 Be  Beryllium      2       2             9.012183   
5                  B      Boron     13       2            10.813500   

               atomic_weight_uncert atomic_weight_data_source  
atomic_number                                                  
1                      1.350000e-04                      nist  
2                      2.000000e-06                      nist  
3                      2.950000e-02                      nist  
4                      5.000000e-07                      nist  
5                      7.500000e-03                      nist  


In [5]:
# Lets put out a table with z, weight in solar masses, z**2 + 5

# First we need to write a query
# Unit conversion is done on the DB side!
from carsus.model import Atom, AtomicWeight
from astropy import units as u
q = session.query(Atom.atomic_number.label("z"),
                  AtomicWeight.quantity.to(u.solMass).value.label("weight_solMass")).\
                  join(Atom.quantities.of_type(AtomicWeight))

# Then we use pandas to read the query into a DataFrame
from pandas import read_sql_query
df = read_sql_query(q.selectable, session.bind)

# Once we have the data we can compute things
df["z**2+5"] = df["z"]**2 + 5

print df.head(5)

   z  weight_solMass  z**2+5
0  1    8.414768e-58       6
1  2    3.341449e-57       9
2  3    5.816602e-57      14
3  4    7.523543e-57      21
4  5    9.027316e-57      30


In [9]:
q.session.query(Atom.atomic_number.label("z"),
                AtomicWeight.quantity.value.label("weight_u")).\
                join(Atom.quantities.of_type(AtomicWeight)).\
                filter(AtomicWeight.quantity < 15*u.u)
df2 = read_sql_query(q.selectable, session.bind)
print df2

     z  weight_solMass
0    1    8.414768e-58
1    2    3.341449e-57
2    3    5.816602e-57
3    4    7.523543e-57
4    5    9.027316e-57
5    6    1.002668e-56
6    7    1.169319e-56
7    8    1.335660e-56
8    9    1.586023e-56
9   10    1.684640e-56
10  11    1.919230e-56
11  12    2.029070e-56
12  13    2.252470e-56
13  14    2.344589e-56
14  15    2.585749e-56
15  16    2.677056e-56
16  17    2.959559e-56
17  18    3.334935e-56
18  19    3.264001e-56
19  20    3.345788e-56
20  21    3.753005e-56
21  22    3.996029e-56
22  23    4.252694e-56
23  24    4.340734e-56
24  25    4.586333e-56
25  26    4.662047e-56
26  27    4.919856e-56
27  28    4.899837e-56
28  29    5.304942e-56
29  30    5.458047e-56
..  ..             ...
63  64    1.312753e-55
64  65    1.326739e-55
65  66    1.356581e-55
66  67    1.376870e-55
67  68    1.396310e-55
68  69    1.410295e-55
69  70    1.444688e-55
70  71    1.460656e-55
71  72    1.490069e-55
72  73    1.510587e-55
73  74    1.534731e-55
74  75    1

In [ ]:
# To store this DataFrame in HDF format we need to create a Dataset object

foo_ds = Dataset("foo", df)

In [ ]:
# create_hdf() is used to put several datasets into an HDF file. Datasets' names are used as the identifiers in the file.

from carsus.io.output import create_hdf
create_hdf("store.h5", datasets=[atoms, foo_ds])


In [ ]:
from pandas import read_hdf
atoms = read_hdf("store.h5", "foo")
print atoms.head(5)